In [3]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
dispatch.dispatch_experiments('feb22', base_job_name='infocompare2', game_name='large_game_2', overrides = overrides, n_seeds=10)
dispatch.dispatch_experiments('feb22', base_job_name='infocompare2', game_name='large_game_2_hide', overrides = overrides, n_seeds=10)


# TODO: RL Learning rate issues???
dispatch.dispatch_experiments('feb22', base_job_name='mar10_very_large_game', game_name='very_large_game_2', overrides = overrides, n_seeds=5)
dispatch.dispatch_experiments('feb22', base_job_name='mar10_large_game_3', game_name='large_game_3', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='tiny_game_Test', game_name='tiny', overrides = overrides, n_seeds=1)

Submitted batch job 7014
Submitted batch job 7015
Submitted batch job 7016
Submitted batch job 7017
Submitted batch job 7018
Submitted batch job 7019
Submitted batch job 7020
Submitted batch job 7021
Submitted batch job 7022
Submitted batch job 7023
Submitted batch job 7024
Submitted batch job 7025
Submitted batch job 7026
Submitted batch job 7027
Submitted batch job 7028
Submitted batch job 7029
Submitted batch job 7030
Submitted batch job 7031
Submitted batch job 7032
Submitted batch job 7033
Submitted batch job 7034
Submitted batch job 7035
Submitted batch job 7036
Submitted batch job 7037
Submitted batch job 7038
Submitted batch job 7039
Submitted batch job 7040
Submitted batch job 7041
Submitted batch job 7042
Submitted batch job 7043


In [ ]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [18]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)
              7014                            large_game_2-feb22mlp-100_infocompare2  R   20:08:32      1 ip-10-255-7-72
              7015                            large_game_2-feb22mlp-101_infocompare2  R   20:05:58      1 ip-10-255-7-66
              7016                            large_game_2-feb22mlp-102_infocompare2  R   20:05:58      1 ip-10-255-7-83
              7019                            large_game_2-feb22mlp-105_infocompare2  R   20:05:58      1 ip-10-255-7-113
              7022                            large_game_2-feb22mlp-108_infocompare2  R   20:05:33      1 ip-10-255-7-67
              7023                            large_game_2-feb22mlp-109_infocompare2  R   20:05:33      1 ip-10-255-7-80
              7024                       large_game_2_hide-feb22mlp-100_infocompare2  R   20:05:33      1 ip-10-255-7-115
              7025          

In [ ]:
# !scancel 5306 5307

In [ ]:
!scancel -u ubuntu

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()